In [5]:
import yfinance as yf

In [8]:
msft = yf.Ticker("MSFT")
msft.history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.055121,0.063227,0.055121,0.060524,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.060524,0.063767,0.060524,0.062686,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.062686,0.064307,0.062686,0.063767,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.063767,0.064307,0.061605,0.062145,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.062145,0.062686,0.060524,0.061065,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2023-07-24 00:00:00-04:00,345.850006,346.920013,342.309998,345.109985,26678100,0.0,0.0
2023-07-25 00:00:00-04:00,347.109985,351.890015,345.070007,350.980011,41637700,0.0,0.0
2023-07-26 00:00:00-04:00,341.440002,344.670013,333.109985,337.769989,58383700,0.0,0.0
